# London at Night
### Follow the Instructions here https://developers.google.com/earth-engine/guides/image_relational#colab-python_1 

In [11]:
import ee
import folium
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# This is needed in python to add layers to Folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        opacity= 0.5,
        control=True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer


Successfully saved authorization token.


In [12]:
# Load a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318')

# Create NDVI and NDWI spectral indices.
ndvi = image.normalizedDifference(['B5', 'B4'])
ndwi = image.normalizedDifference(['B3', 'B5'])

# Create a binary layer using logical operations.
bare = ndvi.lt(0.2).And(ndwi.lt(0))

# Define a map centered on San Francisco Bay.
map_bare = folium.Map(location=[42.3601,-71.0942], zoom_start=12)

# Add the masked image layer to the map and display it.
map_bare.add_ee_layer(bare.selfMask(), None, 'bare')
map_bare

In [13]:
# Load a 2012 nightlights image.
nl_2012 = ee.Image('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182012')
lights = nl_2012.select('stable_lights')

# Define arbitrary thresholds on the 6-bit stable lights band.
zones = lights.gt(30).add(lights.gt(55)).add(lights.gt(62))

# Define a map centered on Paris, France.
map_zones = folium.Map(location=[42.3601,-71.0942], zoom_start=8)

# Display the thresholded image as three distinct zones near Paris.
palette = ['000000', '0000FF', '00FF00', 'FF0000']
map_zones.add_ee_layer(
    zones, {'min': 0, 'max': 3, 'palette': palette, 'opacity':0.5
    }, 'development zones')
map_zones